In [1]:
# Importing dependencies for comunicating with sqlite db
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine,inspect, func, desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import pyodbc
import sqlalchemy as sa
import urllib
import pandas as pd
import os

In [8]:
# Create engine to connect to Sensors.db
engine = create_engine('sqlite:///data/Sensors.db', echo=False)
Base = automap_base()

In [9]:
# Using Base.prepare and Base.classes we will reflect DB into ORM classes
Base.prepare(engine, reflect=True)
Base.classes.keys()

['BME_DATA']

In [10]:
# Explore database with inspector
inspector = inspect(engine)
inspector.get_table_names()

['BME_DATA']

In [11]:
# Save the refference from table BME_DATA
Rambo = Base.classes.BME_DATA

In [12]:
# Create db sesion object
session = Session(engine)

In [13]:
# Creating the query to identify last date to identify end ponint of the year
df = pd.read_sql("SELECT * FROM BME_DATA", engine)

In [14]:
df['TIME_STAMP'] = pd.to_datetime(df['TIME_STAMP'])

In [15]:
# using funciton to convert string into date time
from datetime import datetime, timedelta

def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))


In [16]:
# testing on  single value
test = hour_rounder(df['TIME_STAMP'][0])

In [17]:
# converting fro all values in entire column
df['TIME_STAMP'] = df['TIME_STAMP'].apply(hour_rounder)

In [18]:
# converting temperature from C to f
df['TEMPERATURE'] = round((df['TEMPERATURE']* 9/5) + 32)


In [19]:
# create new tale and drop Id
df = df[["TIME_STAMP", "TEMPERATURE"]]

In [20]:
df.head()

TIME_STAMP  TEMPERATURE
0 2019-04-09 23:00:00        167.0
1 2019-04-09 23:00:00         68.0
2 2019-04-10 00:00:00         68.0
3 2019-04-10 01:00:00         67.0
4 2019-04-10 02:00:00         67.0

In [21]:
# Deleting the wrong input at the first row
df = df.iloc[1:]
df.head()

TIME_STAMP  TEMPERATURE
1 2019-04-09 23:00:00         68.0
2 2019-04-10 00:00:00         68.0
3 2019-04-10 01:00:00         67.0
4 2019-04-10 02:00:00         67.0
5 2019-04-10 03:00:00         66.0

In [22]:
# Importing API data
#API_df = pd.read_csv("data/API.csv")
#API_df.head()

In [23]:
# Merging API and RAMBO dataframes On date
#master_df = pd.merge(fifa_subset, gdp_subset, on = "Nationality", how = "left", suffixes=("_",""))

In [24]:
# Export to csv
df.to_csv("Rambo.csv", sep=',', encoding='utf-8')
df.head()

TIME_STAMP  TEMPERATURE
1 2019-04-09 23:00:00         68.0
2 2019-04-10 00:00:00         68.0
3 2019-04-10 01:00:00         67.0
4 2019-04-10 02:00:00         67.0
5 2019-04-10 03:00:00         66.0

In [2]:
# Importing API data
dfr = pd.read_csv("data/Rambo.csv")
dfr.head()

TIME_STAMP  TEMPERATURE  F_Temp  A_Temp
0  2019-4-11 03:00           70   65.45   65.81
1  2019-4-11 04:00           68   62.79   64.10
2  2019-4-11 05:00           67   60.54   63.04
3  2019-4-11 06:00           67   60.48   62.80
4  2019-4-11 07:00           67   60.08   62.19

In [26]:
# Convert back to string for Json 
dfr['TIME_STAMP'] = dfr['TIME_STAMP'].dt.strftime('%m-%d-%Y %H:%M')
dfr.head()

AttributeError: Can only use .dt accessor with datetimelike values

In [3]:
df_json_ready = dfr.to_json(orient='records')

In [4]:
df_json_ready

'[{"TIME_STAMP":"2019-4-11 03:00","TEMPERATURE":70,"F_Temp":65.45,"A_Temp":65.81},{"TIME_STAMP":"2019-4-11 04:00","TEMPERATURE":68,"F_Temp":62.79,"A_Temp":64.1},{"TIME_STAMP":"2019-4-11 05:00","TEMPERATURE":67,"F_Temp":60.54,"A_Temp":63.04},{"TIME_STAMP":"2019-4-11 06:00","TEMPERATURE":67,"F_Temp":60.48,"A_Temp":62.8},{"TIME_STAMP":"2019-4-11 07:00","TEMPERATURE":67,"F_Temp":60.08,"A_Temp":62.19},{"TIME_STAMP":"2019-4-11 08:00","TEMPERATURE":67,"F_Temp":60.08,"A_Temp":61.62},{"TIME_STAMP":"2019-4-11 09:00","TEMPERATURE":67,"F_Temp":60.88,"A_Temp":60.08},{"TIME_STAMP":"2019-4-11 10:00","TEMPERATURE":69,"F_Temp":60.65,"A_Temp":58.9},{"TIME_STAMP":"2019-4-11 11:00","TEMPERATURE":70,"F_Temp":60.1,"A_Temp":57.95},{"TIME_STAMP":"2019-4-11 12:00","TEMPERATURE":72,"F_Temp":60.14,"A_Temp":58.52},{"TIME_STAMP":"2019-4-11 13:00","TEMPERATURE":74,"F_Temp":63.5,"A_Temp":61.8},{"TIME_STAMP":"2019-4-11 14:00","TEMPERATURE":75,"F_Temp":67.9,"A_Temp":65.99},{"TIME_STAMP":"2019-4-11 15:00","TEMPERATURE"

In [21]:
df.to_json(r'Rambo.json')